In [1]:
import csv
import re
from glob import glob

In [2]:
# All dataset files start with csv
csv_files = [f for f in glob('*.csv') if f.startswith('Dataset')]

# Set the regex pattern for finding URLs
PATTERN = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

# Store a list of URLs with the geolocation they were shared in
website_locations = []

for csv_file in csv_files:
    with open(csv_file) as freader:
        csvreader = csv.reader(freader)
        for line in csvreader:
            urls = PATTERN.findall(line[1])
            for url in urls:
                website_locations.append([url, (line[-4].strip(), line[-3].strip())])

In [3]:
# Find unique websites (NOT domain names)
unique_websites = set()
for website in website_locations:
    unique_websites.add(website[0])

In [8]:
import requests

def get_url(url):
    if websites[url] is not None:
        return
    session = requests.Session()
    try:
        resp = session.head(url.strip(), allow_redirects=True)
        if resp.ok:
            websites[url] = resp.url
    except:
        pass
    
websites = dict.fromkeys(unique_websites, None)

In [6]:
from urllib.parse import urlparse

def get_domain(url):
    parsed_uri = urlparse(url)
    domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return domain

In [9]:
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool

pool = ThreadPool(16)

with tqdm(total=len(websites)) as pbar:
    for i, _ in tqdm(enumerate(pool.imap_unordered(get_url, unique_websites))):
        pbar.update()

0it [00:00, ?it/s]
0it [00:00, ?it/s]



In [10]:
zip2fips = requests.get('https://raw.githubusercontent.com/bgruber/zip2fips/master/zip2fips.json').json()

In [11]:
!pip install uszipcode

from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()

In [13]:
website_fips = {}

for website in website_locations:
    long_url = websites[website[0]]
    if long_url is None:
        continue
    else:
        domain = get_domain(long_url)
        lat, lon = float(website[1][0]), float(website[1][1])
        res = search.by_coordinate(lat, lon)
        if len(res) > 0:
            zipcode = res[0]["Zipcode"]
            if zipcode in zip2fips:
                website_fips.append([domain, zip2fips[zipcode]])